In [132]:
import torch
from sentence_transformers import SentenceTransformer, util
import pandas as pd

In [133]:
crawled_data = pd.read_csv('인공지능_문화체육관광부_클라썸_crawled.csv')

In [134]:
query = '인공지능'
business = '클라썸'

In [135]:
### crawling_trg에 business 이름이 있는 것과 없는 것을 구분하여 데이터를 나눕니다.

crawled_data['checker'] = crawled_data.crawling_trg.str.find(business)

bussiness_news = crawled_data[crawled_data.checker > -1].copy().reset_index(drop=True)
bussiness_news['idx_original'] = range(len(bussiness_news))

org_news = crawled_data[crawled_data.checker == -1].copy().reset_index(drop=True)
org_news['idx_original'] = range(len(org_news))

business_news_contents = bussiness_news[['idx_original', 'crawling_trg', 'pubDate', 'title', 'content', 'link']]
org_news_contents = org_news[['idx_original', 'crawling_trg', 'pubDate', 'title', 'content', 'link']]

In [136]:
### 모델과 토크나이저를 SentenceTransformer 클래스로 불러옵니다.

pretrained_model_path = "sbert/training_klue_sts_klue-roberta-base-2022-08-17_23-27-13"
model = SentenceTransformer(pretrained_model_path)

In [137]:
### 유사도를 계산하여 인덱스를 반환하는 함수입니다.

def retrieve(query, news_contents, top_k):
    query_embedding = model.encode(query)
    news_embeddings = model.encode(news_contents.content)

    cos_scores = util.pytorch_cos_sim(query_embedding, news_embeddings).squeeze()

    top_k = min(top_k, len(news_contents.content))
    top_k_results = torch.topk(cos_scores, k=top_k)
    
    scores = top_k_results.values.squeeze()
    indices = top_k_results.indices.squeeze()

    return indices, scores

In [138]:
### 쿼리와 business 문서 뭉치 간의 유사도 점수를 계산하여 가장 높은 점수의 business 문서를 찾아옵니다.

indices, scores = retrieve(query, business_news_contents, 1)
top_of_business_news_contents = business_news_contents.iloc[int(indices)].to_list() + list([float(scores)])
top_of_business_news_contents = pd.DataFrame([top_of_business_news_contents], columns=['idx_original', 'crawling_trg', 'pubDate', 'title', 'content', 'link', 'score'])

In [139]:
### 기관별로 데이터를 나눕니다.

org_news_contents_splits = []
for org in org_news_contents.crawling_trg.unique():
    org_news_contents_split = org_news_contents[org_news_contents.crawling_trg == org].reset_index(drop=True).copy()
    org_news_contents_splits.append(org_news_contents_split)

In [140]:
### 쿼리와 각 기관별 문서 뭉치간의 유사도 점수를 계산하여 기관별 가장 높은 점수의 문서를 찾아옵니다.

tops_of_org_news_contents_splits = []
for org_news_contents_split in org_news_contents_splits:
    indices, scores  = retrieve(query, org_news_contents_split, 1)
    top_of_org_news_contents_split = org_news_contents_split.iloc[int(indices)].to_list() + list([float(scores)])
    tops_of_org_news_contents_splits.append(top_of_org_news_contents_split)
    
tops_of_org_news_contents_splits = pd.DataFrame(tops_of_org_news_contents_splits, columns=['idx_original', 'crawling_trg', 'pubDate', 'title', 'content', 'link', 'score'])
# tops_of_org_news_contents_splits = tops_of_org_news_contents_splits.sort_values('score', ascending=False)

In [141]:
### 가장 높은 점수의 business 문서를 쿼리로 하여 
### 가장 높은 점수의 기관별 문서 뭉치와 유사도 점수를 계산하고 유사도 점수 상위 5개 문서를 찾아옵니다.

indices, scores = retrieve(top_of_business_news_contents.content, tops_of_org_news_contents_splits, 5)
result = tops_of_org_news_contents_splits.iloc[list(indices)].copy()
result['score'] = scores
# result = result.sort_values('score', ascending=False) ### 이걸 실행했을 때와 안 했을 때 마지막 결과값이 다름.

In [142]:
top_of_business_news_contents.to_csv('기업체_신문기사_중_유사도_점수_최고점인_항목.csv', index=False, encoding='utf-8-sig')
top_of_business_news_contents = pd.read_csv('기업체_신문기사_중_유사도_점수_최고점인_항목.csv')

tops_of_org_news_contents_splits.to_csv('쿼리_기관별_신문기사_유사도_결과.csv', index=False, encoding='utf-8-sig')
tops_of_org_news_contents_splits = pd.read_csv('쿼리_기관별_신문기사_유사도_결과.csv')

result.to_csv('기업체_신문기사와_다시_유사도_점수를_구한_결과.csv', index=False, encoding='utf-8-sig')
result = pd.read_csv('기업체_신문기사와_다시_유사도_점수를_구한_결과.csv')

In [143]:
top_of_business_news_contents

,idx_original,crawling_trg,pubDate,title,content,link,score
0,3,클라썸 인공지능,"Thu, 13 Oct 2022 17:57:00 +0900",D·N·A로 사회적 가치 실현…제3회 소셜DNA혁신상 공모,ICT기술 기반 사회적 가치 실현 우수사례 발굴·확산 일환정보통신기술(ICT)을 바...,https://n.news.naver.com/mnews/article/031/000...,-0.044634


In [144]:
tops_of_org_news_contents_splits

,idx_original,crawling_trg,pubDate,title,content,link,score
0,3,국립국어원 인공지능,"Sat, 08 Oct 2022 10:44:00 +0900","와이즈넛, `세계 한국어 한마당`서 AI 챗봇 체험 행사",와이즈넛은 6~9일 국립한글박물관에서 진행되는 '2022 세계 한국어 한마당 - 한...,https://n.news.naver.com/mnews/article/029/000...,0.184452
1,36,국립민속박물관 인공지능,"Tue, 13 Sep 2022 11:17:00 +0900","국립중앙박물관·전자통신연구원, 지능형 큐레이션 플랫폼 포럼",기사내용 요약박물관서 활용 가능한 문화기술 연구성과 공유[서울=뉴시스] 박진희 기자...,https://n.news.naver.com/mnews/article/003/001...,-0.046029
2,46,국립아시아문화전당 인공지능,"Tue, 27 Sep 2022 19:45:00 +0900",[키워드 브리핑] AICON 광주 2022 등,"- AICON 광주 2022- '2022 청소년과학페어 전국대회', 국제 워크숍 해...",https://n.news.naver.com/mnews/article/660/000...,0.098716
3,56,국립장애인도서관 인공지능,"Thu, 11 Aug 2022 12:45:00 +0900","윤성용 관장 이건희 컬렉션, 시카고 박물관 등 해외 전시 추진",기사내용 요약국립중앙박물관장 취임 후 첫 간담회[서울=뉴시스] 박진희 기자 = 윤성...,https://n.news.naver.com/mnews/article/003/001...,-0.146014
4,57,국립중앙도서관 인공지능,"Sun, 16 Oct 2022 10:02:00 +0900",[2022 기사창투 주역-③]한국전자통신연구원 호남권연구센터,한국전자통신연구원(ETRI) 호남권연구센터는 광주·전남·전북·제주 등 호남광역경제권...,https://n.news.naver.com/mnews/article/030/000...,-0.067610
5,61,국립중앙박물관 인공지능,"Sun, 23 Oct 2022 14:31:00 +0900",전시안내 로봇 나를 따르라,(서울=연합뉴스) 김주성 기자 = 23일 서울 용산구 국립중앙박물관을 찾은 관람객들...,https://n.news.naver.com/mnews/article/001/001...,0.079737
6,84,국립한글박물관 인공지능,"Tue, 11 Oct 2022 10:25:00 +0900","와이즈넛, 2022 세계 한국어 한마당서 인공지능(AI) 챗봇 체험 장 개최",와이즈넛 제공(서울=뉴스1) 임해중 기자 = 인공지능 전문 기업 와이즈넛이 이달 6...,https://n.news.naver.com/mnews/article/421/000...,0.186322
7,112,대한민국역사박물관 인공지능,"Wed, 10 Aug 2022 15:20:00 +0900",우영우가 사랑하는 ‘고래’ 광화문에서 볼 수 있다,광화문 광장 개장...‘광화시대’ 즐기는 방법 인기리에 방영 중인 드라마 ‘이상한 ...,https://n.news.naver.com/mnews/article/025/000...,-0.071729
8,114,한국예술종합학교 인공지능,"Sat, 22 Oct 2022 10:02:00 +0900",안드로이드도 인간처럼 재판받을 권리가 있는가,뮤지컬 ‘인간의 법정’인간의 존엄성. 인간 외에 무엇이 이를 인정했던가. 인간이 인...,https://n.news.naver.com/mnews/article/262/000...,0.041952
9,115,해외문화홍보원 인공지능,"Thu, 11 Aug 2022 10:41:00 +0900","이종호 장관, 외신에 韓 과학기술·디지털 정책 알린다",기사내용 요약외신기자 정책토론회…10개국 외신 참석기술혁신·디지털정책·국가 R&D ...,https://n.news.naver.com/mnews/article/003/001...,-0.009162


In [145]:
result

,idx_original,crawling_trg,pubDate,title,content,link,score
0,46,국립아시아문화전당 인공지능,"Tue, 27 Sep 2022 19:45:00 +0900",[키워드 브리핑] AICON 광주 2022 등,"- AICON 광주 2022- '2022 청소년과학페어 전국대회', 국제 워크숍 해...",https://n.news.naver.com/mnews/article/660/000...,0.651574
1,253,한국문학번역원 인공지능,"Thu, 18 Aug 2022 02:29:00 +0900",8월18일 알림,국보 제332호 수마노탑. 정암사 제공 ◇ 정선 정암사는(주지 천웅 스님)는 18일...,https://n.news.naver.com/mnews/article/028/000...,0.624225
2,57,국립중앙도서관 인공지능,"Sun, 16 Oct 2022 10:02:00 +0900",[2022 기사창투 주역-③]한국전자통신연구원 호남권연구센터,한국전자통신연구원(ETRI) 호남권연구센터는 광주·전남·전북·제주 등 호남광역경제권...,https://n.news.naver.com/mnews/article/030/000...,0.584024
3,393,영화진흥위원회 영화관입장권 통합전산망 인공지능,"Mon, 08 Aug 2022 11:09:00 +0900","[ET라씨로] 개봉 D-2 헌트 예매율 1위…버킷스튜디오, 주가 18% ↑",ET라씨로는 인공지능(AI) 기반으로 선별·분석한 주식 정보와 종목 매매 신호를 제...,https://n.news.naver.com/mnews/article/030/000...,0.556029
4,163,영화진흥위원회 인공지능,"Mon, 08 Aug 2022 11:09:00 +0900","[ET라씨로] 개봉 D-2 헌트 예매율 1위…버킷스튜디오, 주가 18% ↑",ET라씨로는 인공지능(AI) 기반으로 선별·분석한 주식 정보와 종목 매매 신호를 제...,https://n.news.naver.com/mnews/article/030/000...,0.556029
